#### Imports

In [1]:
import logging
import torch

#### Setup logger

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [3]:
logger.info(f'Using torch: {torch.__version__}')

Using torch: 1.4.0


#### Essentials

In [4]:
TRAINING_ID = 'multi-modal-cnn'
DATASET_ROOT = './data/'
CAPTIONS_TRAIN_SET_PATH = 'embedding/caption_embedding.csv'
CAPTIONS_VALIDATION_SET_PATH = 'embedding/caption_embedding.csv'

In [ ]:
EMBEDDING_DIMENSIONALITY = 1000 # number of CNN outputs = dimensionality of the captions word2vec model
BATCH_SIZE = 64 
EPOCHS = 10
PRINT_FREQ = 1 # how frequently to print loss value to screen
PLOT = True  # save a plot with the training and validation losses
WORKERS = 64 # number of data loading workers = CPU cores
GPU = 0
LR = 0.01 
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4